In [32]:
# %pip install gensim --user

In [33]:
import gensim

In [34]:
import sys, os
# sys.path.insert(0, '/datasets/home/51/451/yuz530/.local/lib/python3.7/site-packages/')

In [35]:
!ls .

ABA.cor             data                jeff_word2vec.ipynb test.cor


In [36]:
import json
from scipy import sparse
import numpy as np
from gensim.test.utils import datapath
from gensim import utils
import gensim.models
from tqdm import tqdm
from node2vec import Node2Vec
import networkx as nx
from itertools import cycle
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction

In [45]:
A = sparse.load_npz('./data/A.npz')
B = sparse.load_npz('./data/B.npz')
P = sparse.load_npz('./data/P.npz')
A_csr = A
A_csc = A.tocsc(copy=True)  # memory is cheap ;D

In [11]:
# ABPBA
def ABPBA():
    while True:
        app_i = np.random.choice(np.arange(A.shape[0]))
        api_i = np.random.choice(np.nonzero(A_csr[app_i])[1])
        api_bi = np.random.choice(np.nonzero(B[:, api_i])[0])
        api_p = np.random.choice(np.nonzero(P[:, api_bi])[0])
        api_bj = np.random.choice(np.nonzero(B[:, api_p])[0])
        app_j = np.random.choice(np.nonzero(A_csc[:, api_bj])[0])

        yield f'app_{app_i} api_{api_i} api_{api_bi} api_{api_p} api_{api_bj} app_{app_j}'
        
# ABA
def ABA():
    while True:
        app_i = np.random.choice(np.arange(A.shape[0]))
        api_i = np.random.choice(np.nonzero(A_csr[app_i])[1])
        api_bi = np.random.choice(np.nonzero(B[:, api_i])[0])
        app_j = np.random.choice(np.nonzero(A_csc[:, api_bi])[0])

        yield f'app_{app_i} api_{api_i} api_{api_bi} app_{app_j}'
        
# it = iter_neighbors()

In [12]:
app_i = np.random.choice(np.arange(A.shape[0]))
print(app_i)
api_i = np.random.choice(np.nonzero(A_csr[app_i])[1])
print(api_i)
api_bi = np.random.choice(np.nonzero(B[:, api_i])[0])
print(api_bi)
if(api_bi > A.shape[1]):
    api_bi = A.shape[1]-1
app_j = np.random.choice(np.nonzero(A_csc[:, api_bi])[0])
app_j

102
12338
699629


73

In [13]:
(B[:, api_i])

<916573x1 sparse matrix of type '<class 'numpy.int64'>'
	with 747 stored elements in Compressed Sparse Row format>

In [14]:
# %lprun -f iter_neighbors a = [next(it) for i in range(10000)] 

In [15]:
#note ValueError: 'a' cannot be empty unless no samples are taken B col all 0 why?

for _ in range(10):
    print(next(ABA()) + '\n')

ValueError: 'a' cannot be empty unless no samples are taken

In [115]:
import n2v
import numpy as np
n2v.node2vec(p=2, q=1, walk_length=20)
# n2v.test()

NameError: name 'np' is not defined

In [102]:
n2v

<module 'n2v' from '/Users/jeffliao/Desktop/HinDroid-with-Embeddings/notebooks/jeff_eda/n2v.py'>

In [79]:
' '.join(n2v.node2vec(p=2, q=1, walk_length=20))

TypeError: 'module' object is not callable

In [69]:
%%time
f = open('n2v.cor', 'w')
for _ in range(10000):
    try:
        f.write(' '.join(node2vec(p=2, q=1, walk_length=20)) + '\n')
    except:
        pass
f.close()

CPU times: user 3h 23min 2s, sys: 9min 43s, total: 3h 32min 45s
Wall time: 1h 22min 46s


In [70]:
!head n2v.cor

app_417 api_184527 app_328 api_241420 app_168 api_184528 api_241412 api_241411 app_414 api_241411 api_184532 api_184534 api_392214 app_49 api_392214 api_1205262 api_392214 api_1204460 api_1204459 api_1205273
app_648 api_1404756 api_1437613 api_1404923 api_1474609 api_1474616 api_1474600 api_1870 api_1404756 api_1870 api_1410542 api_1 api_1410542 api_1 api_1534306 app_909 api_1534302 api_1 api_9072 api_1187199
app_274 api_31276 api_38217 api_1835 api_38217 api_230660 api_116196 app_417 api_230759 app_214 api_230759 app_210 api_230759 api_1780 api_81 api_361800 api_537 api_1263 api_188981 app_15
app_251 api_946212 api_946213 api_946212 api_946209 app_251 api_95413 app_65 api_464224 api_44888 api_44390 app_28 api_302642 api_1195809 api_302642 api_1195809 api_1196722 api_1142 api_1196722 api_1196796
app_580 api_1411839 app_580 api_1411829 app_580 api_1411632 app_580 api_1411957 api_1411958 api_1411808 api_1411958 api_1411808 app_580 api_1412004 api_6000 api_26172 api_595538 api_595875 api_

In [51]:
# f = open('test.cor', 'w')
# for _ in range(10000):
#     f.write(next(it) + '\n')
# f.close()

In [52]:
!head ABA.cor

In [121]:
%%time
import w2v
model = w2v.create_model()

CPU times: user 28 µs, sys: 0 ns, total: 28 µs
Wall time: 31 µs


In [122]:
model

In [71]:
%%time
from gensim import utils
import gensim.models

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'n2v.cor'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
#             yield utils.simple_preprocess(line)
            yield line.strip().split(' ')

CPU times: user 44 µs, sys: 38 µs, total: 82 µs
Wall time: 86.1 µs


In [72]:
%%time
sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, min_count=1, size=200)

/Users/jeffliao/.local/lib/python3.7/site-packages/gensim/models/base_any2vec.py:743: UserWarning:

C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.



CPU times: user 10min 19s, sys: 1min 58s, total: 12min 18s
Wall time: 8min 4s


In [73]:
for i, word in enumerate(model.wv.vocab):
    if i == 10:
        break
    print(word)

app_417
api_184527
app_328
api_241420
app_168
api_184528
api_241412
api_241411
app_414
api_184532


In [74]:
model.wv.similarity("app_177", "app_32")

0.17143801

In [75]:
print(model.wv.most_similar(positive=['app_177'], topn=5))

[('api_403', 0.3776176869869232), ('app_478', 0.37359991669654846), ('api_1942', 0.3616974651813507), ('api_423', 0.35740453004837036), ('api_245', 0.3544881343841553)]


In [76]:
def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.vocab:
        if 'app' in word:
            if int(word.split('_')[1]) > 332:
                labels.append(1)
            else:
                labels.append(0)

            vectors.append(model.wv[word])
            #labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels



def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='markers', text=labels, marker=dict(size=5, color=labels))

    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))
    plt.show()

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

In [77]:
x_vals, y_vals, labels = reduce_dimensions(model)

plot_with_plotly(x_vals, y_vals, labels)
